In [1]:
#importing and exporting data modules
from simple_salesforce import Salesforce
import gspread
import requests
from io import StringIO

import pandas as pd
import numpy as np
from datetime import datetime, timedelta

import matplotlib.pyplot as plt


In [2]:
sf = Salesforce(username='bchen@envoy.com',password='TasksandEvents1', security_token='nQWlT8vNdnJwxwtfpS1ic4Z7O')
gc = gspread.service_account(filename='/Users/bchen/Downloads/gspread-428120-06947c66447d.json')
sht1 = gc.open_by_key('1A6Q8dvoWwLi26tnQQoEo6ZnekvZBQbpgCyP20iQV6Ug')


In [3]:
sf_instance = 'https://envoy.my.salesforce.com/' #Your Salesforce Instance URL
reportId = '00OUO000009IZVD2A4' # add report id
export = '?isdtp=p1&export=1&enc=UTF-8&xf=csv'
sfUrl = sf_instance + reportId + export
response = requests.get(sfUrl, headers=sf.headers, cookies={'sid': sf.session_id})
download_report = response.content.decode('utf-8')
df = pd.read_csv(StringIO(download_report))

In [4]:


# Your existing functions:
def get_quarter_info(current_date):
    fiscal_year = current_date.year if current_date.month >= 2 else current_date.year - 1

    if datetime(current_date.year, 2, 1) <= current_date < datetime(current_date.year, 5, 1):
        quarter = f"{fiscal_year + 1}-Q1"
        quarter_start_date = datetime(current_date.year, 2, 1)
        quarter_end_date = datetime(current_date.year, 4, 30)
    elif datetime(current_date.year, 5, 1) <= current_date < datetime(current_date.year, 8, 1):
        quarter = f"{fiscal_year + 1}-Q2"
        quarter_start_date = datetime(current_date.year, 5, 1)
        quarter_end_date = datetime(current_date.year, 7, 31)
    elif datetime(current_date.year, 8, 1) <= current_date < datetime(current_date.year, 11, 1):
        quarter = f"{fiscal_year + 1}-Q3"
        quarter_start_date = datetime(current_date.year, 8, 1)
        quarter_end_date = datetime(current_date.year, 10, 31)
    else:
        # Q4: Ends on Jan 31 of the following year
        if current_date.month == 1:
            quarter = f"{fiscal_year + 1}-Q4"
            quarter_start_date = datetime(current_date.year - 1, 11, 1)
            quarter_end_date = datetime(current_date.year, 1, 31)
        else:
            quarter = f"{fiscal_year + 1}-Q4"
            quarter_start_date = datetime(current_date.year, 11, 1)
            quarter_end_date = datetime(current_date.year + 1, 1, 31)
    return quarter, quarter_start_date, quarter_end_date

def compute_week_boundaries_for_quarter(quarter_start_date, quarter_end_date):
    total_days = (quarter_end_date - quarter_start_date).days + 1
    start_wd = quarter_start_date.weekday()  # Monday=0,...,Sunday=6

    # Fixed 11 weeks in the middle: 11*7 = 77 days
    leftover = total_days - 77
    solutions = []
    half = leftover / 2.0

    for W1 in range(2, 13):
        W13 = leftover - W1
        if 4 <= W13 <= 12 and ((start_wd + W1) % 7 == 4):
            solutions.append((W1, W13))

    if not solutions:
        raise ValueError(f"Could not find a valid week distribution for quarter starting {quarter_start_date}.")

    solutions.sort(key=lambda x: abs(x[0] - half))
    best_W1, best_W13 = solutions[0]

    week_lengths = [best_W1] + [7]*11 + [best_W13]
    week_boundaries = []
    start_of_week = quarter_start_date
    for length in week_lengths:
        end_of_week = start_of_week + timedelta(days=length - 1)
        week_boundaries.append((start_of_week, end_of_week))
        start_of_week = end_of_week + timedelta(days=1)
    return week_boundaries

# Helper function to break down a single date
def breakdown_date(date_val):
    if pd.isnull(date_val):
        return pd.Series({
            'Quarter': None,
            'Week_of_Quarter': None,
            'Month': None,
            'Day_of_Week': None,
            'Day_Name': None
        })
    # Use your custom logic for quarter and week
    quarter, q_start, q_end = get_quarter_info(date_val)
    week_boundaries = compute_week_boundaries_for_quarter(q_start, q_end)
    week_of_quarter = None
    for i, (wstart, wend) in enumerate(week_boundaries):
        if wstart <= date_val <= wend:
            week_of_quarter = i + 1
            break
    return pd.Series({
        'Quarter': quarter,
        'Week_of_Quarter': week_of_quarter,
        'Month': date_val.month,
        'Day_of_Week': date_val.weekday(),       # Monday=0, Sunday=6
        'Day_Name': date_val.strftime("%A")
    })

# Assume you have loaded your SFDC report into a DataFrame called df
# For example: df = pd.read_csv("sfdc_report.csv")

# List of date columns to enhance
date_columns = ['Created Date', 'SQO Date', 'SAO Date', 'Timestamp: Solution Validation', 'Close Date']

# Convert each to datetime
for col in date_columns:
    df[col] = pd.to_datetime(df[col], errors='coerce')

# Process each date column and merge the breakdown back into df
for col in date_columns:
    # Apply the breakdown function to the column
    breakdown_df = df[col].apply(breakdown_date)
    # Rename the resulting columns to include the original column name as prefix
    breakdown_df = breakdown_df.rename(columns={
        'Quarter': f'{col}_Quarter',
        'Week_of_Quarter': f'{col}_Week_of_Quarter',
        'Month': f'{col}_Month',
        'Day_of_Week': f'{col}_Day_of_Week',
        'Day_Name': f'{col}_Day_Name'
    })
    # Concatenate the breakdown info back into the main DataFrame
    df = pd.concat([df, breakdown_df], axis=1)

# At this point, df contains your original columns plus the breakdowns for each date field.
# You can then save your master report:
df.to_csv("master_report.csv", index=False)


In [5]:
worksheet = sht1.worksheet("Data")

# Reset the index to include the "Assigned" names as a column
data_to_update = df.copy()

# Convert column headers to strings (if they are not already)
data_to_update.columns = data_to_update.columns.astype(str)

# Convert any Pandas Timestamp in the dataframe to a string
data_to_update = data_to_update.applymap(
    lambda x: x.strftime('%Y-%m-%d') if isinstance(x, pd.Timestamp) else x
)

# Replace NaN, inf, and -inf with empty strings
import numpy as np
data_to_update = data_to_update.fillna('')
data_to_update = data_to_update.replace([np.inf, -np.inf], '')

# Prepare the data as a list of lists (headers + rows)
values = [data_to_update.columns.values.tolist()] + data_to_update.values.tolist()

# Update the Google Sheet by passing the values first and the range second
worksheet.update(values, "A1")

/var/folders/89/mmk3x71s01g09b0bf8137yrw0000gp/T/ipykernel_23761/3297849996.py:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_to_update = data_to_update.applymap(


{'spreadsheetId': '1A6Q8dvoWwLi26tnQQoEo6ZnekvZBQbpgCyP20iQV6Ug',
 'updatedRange': 'Data!A1:BF13065',
 'updatedRows': 13065,
 'updatedColumns': 58,
 'updatedCells': 757770}

In [6]:
def get_last_completed_quarters(n, today=None):
    """
    Returns a list of the last n completed quarters (as strings) based on the custom fiscal calendar.
    For example, if today is 03/06/2025 (current quarter = "2026-Q1"), the most recent completed quarter
    is "2025-Q4".
    """
    if today is None:
        today = datetime.today()
    # Determine the current quarter (which is in progress)
    current_quarter, q_start, q_end = get_quarter_info(today)
    # Last completed quarter: one day before current quarter starts.
    last_date = q_start - timedelta(days=1)
    quarters = []
    for _ in range(n):
        q, q_start, q_end = get_quarter_info(last_date)
        quarters.append(q)
        last_date = q_start - timedelta(days=1)
    # Reverse to show them in chronological order (oldest left, newest right)
    return quarters[::-1]

# ---------------------------
# Define dynamic parameters and filter the data
# ---------------------------
# Set "today" dynamically
today = datetime.today()
last_5_quarters = get_last_completed_quarters(5, today)
print("Last 5 completed quarters:", last_5_quarters)

# Define the three valid segments
valid_segments = ['Enterprise', 'Mid Market', 'SMB']

# Filter the master DataFrame for the segments of interest
df_filtered = df[df['Segment - historical'].isin(valid_segments)]

# For bookings, we only want opportunities with Stage = "Closed Won"
df_bookings = df_filtered[df_filtered['Stage'] == 'Closed Won']

# Ensure that the "Close Date_Quarter" column exists.
# If it doesn’t, create it using get_quarter_info on the "Close Date" column.
if 'Close Date_Quarter' not in df_bookings.columns:
    def get_close_quarter(date_val):
        if pd.isnull(date_val):
            return None
        q, _, _ = get_quarter_info(date_val)
        return q
    df_bookings['Close Date_Quarter'] = df_bookings['Close Date'].apply(get_close_quarter)

# Now, filter df_bookings to only include rows from the last 5 completed quarters
df_bookings = df_bookings[df_bookings['Close Date_Quarter'].isin(last_5_quarters)]

# ---------------------------
# View 1: Bookings by Quarter (by Segment)
# ---------------------------
view1 = pd.pivot_table(
    df_bookings,
    index='Segment - historical',
    columns='Close Date_Quarter',
    values='ARR Change',
    aggfunc='sum',
    fill_value=0
)
# Reorder columns to match last_5_quarters
view1 = view1.reindex(columns=last_5_quarters)
# Add a "Total" row that sums each quarter
view1.loc['Total'] = view1.sum()

# ---------------------------
# View 2: Bookings Type Breakdown (Gross and %)
# ---------------------------
# Pivot table with rows = Bookings Type, columns = Close Date_Quarter, values = sum of ARR Change
view2_gross = pd.pivot_table(
    df_bookings,
    index='Bookings Type',
    columns='Close Date_Quarter',
    values='ARR Change',
    aggfunc='sum',
    fill_value=0
)
view2_gross = view2_gross.reindex(columns=last_5_quarters)

# Create a percentage view (each quarter column sums to 100)
view2_pct = view2_gross.div(view2_gross.sum(axis=0), axis=1) * 100
view2_pct = view2_pct.round(2)

# ---------------------------
# View 3: Bookings Type by Segment (Gross and %)
# ---------------------------
# Pivot table with a multi-index: rows = [Segment, Bookings Type], columns = Close Date_Quarter
view3_gross = pd.pivot_table(
    df_bookings,
    index=['Segment - historical', 'Bookings Type'],
    columns='Close Date_Quarter',
    values='ARR Change',
    aggfunc='sum',
    fill_value=0
)
view3_gross = view3_gross.reindex(columns=last_5_quarters)

# Create a percentage view.
# For each segment, for each quarter column, calculate the percentage breakdown by Bookings Type.
view3_pct = view3_gross.copy()
for seg in view3_pct.index.get_level_values(0).unique():
    idx = view3_pct.index.get_level_values(0) == seg
    view3_pct.loc[idx] = view3_pct.loc[idx].div(view3_pct.loc[idx].sum(), axis=1) * 100
view3_pct = view3_pct.round(2)

Last 5 completed quarters: ['2024-Q4', '2025-Q1', '2025-Q2', '2025-Q3', '2025-Q4']


In [7]:
view1

Close Date_Quarter,2024-Q4,2025-Q1,2025-Q2,2025-Q3,2025-Q4
Segment - historical,,,,,
Enterprise,1895396.88,1744913.53,1466456.74,1198163.71,1678246.63
Mid Market,926954.65,993827.14,701977.96,838328.05,615796.10
SMB,769214.99,725884.30,916089.91,884685.97,873751.14
Total,3591566.52,3464624.97,3084524.61,2921177.73,3167793.87


In [8]:
view2_gross

Close Date_Quarter,2024-Q4,2025-Q1,2025-Q2,2025-Q3,2025-Q4
Bookings Type,,,,,
Expansion,2466665.80,2054268.98,1997575.13,2038122.63,1812195.18
New Business,1124900.72,1410355.99,1086949.48,883055.10,1355598.69


In [9]:
view2_pct

Close Date_Quarter,2024-Q4,2025-Q1,2025-Q2,2025-Q3,2025-Q4
Bookings Type,,,,,
Expansion,68.68,59.29,64.76,69.77,57.21
New Business,31.32,40.71,35.24,30.23,42.79


In [10]:
view3_gross

Close Date_Quarter                     2024-Q4    2025-Q1    2025-Q2  \
Segment - historical Bookings Type                                     
Enterprise           Expansion      1450446.96  978824.55  940181.38   
                     New Business    444949.92  766088.98  526275.36   
Mid Market           Expansion       619254.21  654186.94  421548.27   
                     New Business    307700.44  339640.20  280429.69   
SMB                  Expansion       396964.63  421257.49  635845.48   
                     New Business    372250.36  304626.81  280244.43   

Close Date_Quarter                    2025-Q3     2025-Q4  
Segment - historical Bookings Type                         
Enterprise           Expansion      952861.10  1119212.78  
                     New Business   245302.61   559033.85  
Mid Market           Expansion      601870.91   370090.28  
                     New Business   236457.14   245705.82  
SMB                  Expansion      483390.62   322892.12  
                     New Business   401295.35   550859.02

In [11]:
view3_pct

Close Date_Quarter                  2024-Q4  2025-Q1  2025-Q2  2025-Q3  \
Segment - historical Bookings Type                                       
Enterprise           Expansion        76.52    56.10    64.11    79.53   
                     New Business     23.48    43.90    35.89    20.47   
Mid Market           Expansion        66.81    65.83    60.05    71.79   
                     New Business     33.19    34.17    39.95    28.21   
SMB                  Expansion        51.61    58.03    69.41    54.64   
                     New Business     48.39    41.97    30.59    45.36   

Close Date_Quarter                  2025-Q4  
Segment - historical Bookings Type           
Enterprise           Expansion        66.69  
                     New Business     33.31  
Mid Market           Expansion        60.10  
                     New Business     39.90  
SMB                  Expansion        36.95  
                     New Business     63.05

In [12]:
view3_pct

Close Date_Quarter                  2024-Q4  2025-Q1  2025-Q2  2025-Q3  \
Segment - historical Bookings Type                                       
Enterprise           Expansion        76.52    56.10    64.11    79.53   
                     New Business     23.48    43.90    35.89    20.47   
Mid Market           Expansion        66.81    65.83    60.05    71.79   
                     New Business     33.19    34.17    39.95    28.21   
SMB                  Expansion        51.61    58.03    69.41    54.64   
                     New Business     48.39    41.97    30.59    45.36   

Close Date_Quarter                  2025-Q4  
Segment - historical Bookings Type           
Enterprise           Expansion        66.69  
                     New Business     33.31  
Mid Market           Expansion        60.10  
                     New Business     39.90  
SMB                  Expansion        36.95  
                     New Business     63.05

In [13]:
import streamlit as st
import pandas as pd

# Assuming df_bookings is already defined and filtered

# Define your options
metrics = ['ARR Change', 'Bookings Amount']
row_options = ['Segment - historical', 'Bookings Type', 'Source']
column_options = ['Close Date_Quarter', 'Created Date_Quarter']

# Create selection widgets in Streamlit
metric = st.selectbox('Select Metric', metrics)
row_field = st.selectbox('Select Row Field', row_options)
column_field = st.selectbox('Select Column Field', column_options)

# Create a pivot table based on the selections
pivot_df = pd.pivot_table(
    df_bookings, 
    index=row_field, 
    columns=column_field, 
    values=metric, 
    aggfunc='sum', 
    fill_value=0
)

st.dataframe(pivot_df)


2025-03-31 09:26:38.666 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-31 09:26:38.667 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-31 09:26:38.667 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-31 09:26:38.667 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-31 09:26:38.667 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`
2025-03-31 09:26:38.667 WARNING streamlit.runtime.scriptrunner_utils.script_run_c

DeltaGenerator()